## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [8]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="xrp"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('xrp_train_7_3.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,0.20254,0.21184,0.20132,0.21090,1662996864,USD
1,2020-06-02,0.21090,0.21452,0.19826,0.20317,2046863744,USD
2,2020-06-03,0.20316,0.20475,0.20104,0.20411,1226735360,USD
3,2020-06-04,0.20411,0.20713,0.20121,0.20438,1357705728,USD
4,2020-06-05,0.20438,0.20567,0.20174,0.20282,1165720704,USD
...,...,...,...,...,...,...,...
507,2021-10-21,1.14380,1.16343,1.08712,1.08836,619769472,USD
508,2021-10-22,1.08836,1.12831,1.07813,1.08919,475461472,USD
509,2021-10-23,1.08903,1.10149,1.07665,1.09354,263599872,USD
510,2021-10-24,1.09354,1.09548,1.05684,1.08235,315373280,USD


Chuẩn hóa dữ liệu

In [9]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Close'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Close'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Close'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Close'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Close'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}7_3_standardized.csv")
print(df)

           Date     Open     High      Low    Close      Volume Currency  \
20   2020-06-21  0.18793  0.18840  0.18577  0.18606   623938624      USD   
21   2020-06-22  0.18606  0.19018  0.18584  0.18941  1066312000      USD   
22   2020-06-23  0.18942  0.19042  0.18839  0.18886   793852224      USD   
23   2020-06-24  0.18886  0.19010  0.18179  0.18351  1010395648      USD   
24   2020-06-25  0.18352  0.18406  0.17897  0.18155   912398528      USD   
..          ...      ...      ...      ...      ...         ...      ...   
507  2021-10-21  1.14380  1.16343  1.08712  1.08836   619769472      USD   
508  2021-10-22  1.08836  1.12831  1.07813  1.08919   475461472      USD   
509  2021-10-23  1.08903  1.10149  1.07665  1.09354   263599872      USD   
510  2021-10-24  1.09354  1.09548  1.05684  1.08235   315373280      USD   
511  2021-10-25  1.08231  1.10274  1.07836  1.09527   302260768      USD   

         H-L      O-C     SMA_7    SMA_14    SMA_21      SD_7     SD_21  
20   0.00263 

Scale Miền giá trị lại

In [10]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Close']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[6.78916863e-03],
       [8.80545541e-03],
       [8.47442325e-03],
       [5.25438317e-03],
       [4.07470493e-03],
       [4.63445022e-03],
       [2.16675595e-04],
       [1.26394097e-03],
       [1.48663533e-03],
       [3.25013392e-04],
       [1.00513401e-03],
       [0.00000000e+00],
       [9.75040175e-04],
       [1.98619295e-03],
       [1.40237260e-03],
       [8.40821682e-03],
       [6.22340458e-03],
       [1.84535381e-02],
       [1.67502272e-02],
       [1.45654150e-02],
       [1.58714873e-02],
       [1.58955624e-02],
       [1.44089270e-02],
       [1.46436589e-02],
       [1.35121308e-02],
       [1.18148387e-02],
       [1.18569700e-02],
       [1.51131227e-02],
       [1.48964471e-02],
       [1.19833641e-02],
       [1.48603345e-02],
       [1.73220100e-02],
       [2.03012995e-02],
       [1.75928545e-02],
       [2.39847846e-02],
       [2.43639668e-02],
       [2.98229881e-02],
       [3.34823981e-02],
       [4.11743817e-02],
       [4.17642208e-02],


In [11]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [12]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}73.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 41s 62ms/step - loss: 0.0229
Epoch 2/60
40/40 [==============================] - 2s 62ms/step - loss: 0.0135
Epoch 3/60
40/40 [==============================] - 2s 60ms/step - loss: 0.0127
Epoch 4/60
40/40 [==============================] - 2s 62ms/step - loss: 0.0095
Epoch 5/60
40/40 [==============================] - 2s 60ms/step - loss: 0.0133
Epoch 6/60
40/40 [==============================] - 3s 63ms/step - loss: 0.0106
Epoch 7/60
40/40 [==============================] - 3s 66ms/step - loss: 0.0072
Epoch 8/60
40/40 [==============================] - 2s 62ms/step - loss: 0.0083
Epoch 9/60
40/40 [==============================] - 2s 61ms/step - loss: 0.0057
Epoch 10/60
40/40 [==============================] - 2s 51ms/step - loss: 0.0046
Epoch 11/60
40/40 [==============================] - 2s 49ms/step - loss: 0.0054
Epoch 12/60
40/40 [==============================] - 2s 53ms/step - loss: 0.0059
Epoch 13/60
40/40 [=================